<h2>Maven Rewards Challenge:</h2>

<h3>Challenge Objective:</h3>
For the Maven Rewards Challenge, you’ll play the role of a Sr. Marketing Analyst at Maven Cafe.

You've just run a test by sending different combinations of promotional offers to existing rewards members. Now that the 30-day period for the test has concluded, your task is to identify key customer segments and develop a data-driven strategy for future promotional messaging & targeting.

The results need to be summarized in a report that will be presented to the CMO.

<h3>Cafe Rewards Offers:</h3>
Data that simulates the behavior of Cafe Rewards members over a 30-day period, including their transactions and responses to promotional offers.

Customers receive offers once every few days and have a limited time to redeem them. These can be informational offers (simple advertisement of a product), discount offers, or "buy one, get one" (BOGO) offers. Each customer receives a different mix of offers, attempting to maximize their probability of making a purchase.

Every customer purchase during the period is marked as a transaction. For a transaction to be attributed to an offer, it must occur at the same time as when the offer was "completed" by the customer.

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# read all the datasets
customers = pd.read_csv("customers.csv")
events = pd.read_csv("events.csv")
offers = pd.read_csv("offers.csv")

In [3]:
# No. of records and features in each dataset
print("Customers data:", customers.shape)
print("Events data:", events.shape)
print("Offers data:", offers.shape)

Customers data: (17000, 5)
Events data: (306534, 4)
Offers data: (10, 6)


<h4>Data Dictionary:</h4>

<b>Customers: </b>Demographic data for each member

 - customer_id: Unique customer ID (primary key)
 - became_member_on: Date when the customer created their account (yyyymmdd)
 - gender: Customer's gender: (M)ale, (F)emale, or (O)ther
 - age: Customer's age
 - income: Customer's estimated annual income, in USD

<b>Events: </b>Data on customer activity, with records for transactions, offers received, offers viewed, and offers completed

 - customer_id: Customer the event is associated with  (foreign key)
 - event: Description of the event (transaction, offer received, offer viewed, or offer completed)
 - value: Dictionary of values associated with the event (amount for transactions, offer_id for offers received and viewed, and offer_id & reward for offers completed).
 - time: Hours passed in the 30-day period (starting at 0)

<b>Offers: </b>Details on the offers sent to customers during the 30-day period

 - offer_id: Unique offer ID (primary key)
 - offer_type: type of offer: bogo (buy one, get one), discount, or informational
 - difficulty: minimum amount required to spend in order to be able to complete the offer
 - reward: reward (in dollars) obtained by completing the offer
 - duration: days a customer has to complete the offer once they have received it
 - channels: list of marketing channels used to send the offer to customers

In [4]:
# Customer data
customers.head()

,customer_id,became_member_on,gender,age,income
0,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,118,NaN
1,0610b486422d4921ae7d2bf64640c50b,20170715,F,55,112000.0
2,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN,118,NaN
3,78afa995795e4d85b5d9ceeca43f5fef,20170509,F,75,100000.0
4,a03223e636434f42ac4c3df47e8bac43,20170804,NaN,118,NaN


In [5]:
# Events data
events.head()

,customer_id,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [6]:
# Offers data
offers.head()

,offer_id,offer_type,difficulty,reward,duration,channels
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7,"['email', 'mobile', 'social']"
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5,"['web', 'email', 'mobile', 'social']"
2,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"['web', 'email', 'mobile']"
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7,"['web', 'email', 'mobile']"
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10,"['web', 'email']"
